# Unimodal

## Requirements

Given a query, these retrieval systems selects the tracks that are most similar to the query track. The similarity is measured as cosine similarity between the feature of the query and the feature of the target track:
$$ sim(query, target) = cos(feature(query), feature(target)) $$

You will use the following features, hence implementing three different unimodal retrieval systems:

* Lyrics: `id_lyrics_bert_mmsr.tsv`
* Audio:`id_mfcc_bow_mmsr.tsv`
* Videoclips: `id_vgg19_mmsr.tsv`

In [8]:
import pandas as pd
import torch
import os, sys
from tqdm import tqdm

target_folder = 'MMSR25-26-Group-E'
current_path = os.getcwd()

while os.path.basename(current_path) != target_folder:
    if os.path.basename(current_path) == 'RetrievalAlgorithm':
        if os.path.join(current_path) not in sys.path:
            sys.path.append(os.path.join(current_path))
    parent = os.path.dirname(current_path)
    os.chdir(parent)
    current_path = parent

from RetrievalAlgorithm.src.utils.data_loading import load_all_tsv_files_from_path
from RetrievalAlgorithm.src.score_calculation_modules.cosine_similarity_module import CosineSimilarityModule
from RetrievalAlgorithm.src.unimodal_calculations import calculate_unimodal_similarity

## Module initialization

In [9]:
unimodal_module_path = 'RetrievalAlgorithm/modules/cosine_similarity_module.pt'

if os.path.exists(unimodal_module_path):
    unimodal_module = torch.jit.load(unimodal_module_path)
else:
    unimodal_module = CosineSimilarityModule()
    unimodal_module = torch.jit.script(unimodal_module)
    unimodal_module.save(unimodal_module_path)

## Data Loading

In [10]:
dataset_files_dict = load_all_tsv_files_from_path(path_to_dataset='Dataset')

Loading .tsv files: 100%|██████████| 9/9 [00:00<00:00, 692.97it/s]


## Calculating Unimodal Similarity Scores

### Lyrics

In [11]:
lyrics_similarity_scores_df = calculate_unimodal_similarity(
    dataset_df=dataset_files_dict['id_lyrics_bert_mmsr.tsv'],
    calculation_module=unimodal_module,
    feature_name='Lyrics',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [02:05<00:00, 66.89it/s] 


### Audio

In [12]:
audio_similarity_scores_df = calculate_unimodal_similarity(
    dataset_df=dataset_files_dict['id_mfcc_bow_mmsr.tsv'],
    calculation_module=unimodal_module,
    feature_name='Audio',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [01:50<00:00, 76.32it/s] 


### Videoclips

In [13]:
videoclips_similarity_scores_df = calculate_unimodal_similarity(
    dataset_df=dataset_files_dict['id_vgg19_mmsr.tsv'],
    calculation_module=unimodal_module,
    feature_name='Video',
    batch_size=1024,
    include_reverse_pairs=False,
    include_self_pairs=False,
)

Batches: 100%|██████████| 8400/8400 [09:12<00:00, 15.21it/s]


## Merge and Save results

In [14]:
sim_scores_df_list = [
    ('lyrics', lyrics_similarity_scores_df),
    ('audio', audio_similarity_scores_df),
    ('videoclips', videoclips_similarity_scores_df)
]

target_dir = 'RetrievalAlgorithm/results/unimodal/raw'
os.makedirs(target_dir, exist_ok=True)

for feature_name, sim_scores_df in tqdm(sim_scores_df_list, desc='Saving similarity scores'):
    output_path = os.path.join(target_dir, f'unimodal_{feature_name}_similarity_scores.parquet')
    sim_scores_df.to_parquet(output_path, index=False)

Saving similarity scores: 100%|██████████| 3/3 [00:09<00:00,  3.22s/it]
